# Assignment #1: KNN Classifier


In [135]:
pip install ucimlrepo

Note: you may need to restart the kernel to use updated packages.


In [136]:
import numpy as np
import numpy.typing as npt
from ucimlrepo import fetch_ucirepo

### 1. Write a function to calculate and return the Minkowski distance with optional argument p defaulting to ‘p=2’ (Euclidean) of two vectors where a vector represents a data point.

Note: Minkowski Distance is the generalized form of distance calculations with p=1 representing Manhattan distance and p=2 representing Euclidean

In [137]:
def minkowski_dist(v1: np.ndarray, v2: np.ndarray, p: int=2):
    return np.sum((np.absolute(np.power(np.subtract(v1, v2), p))))**(1.0/p)

In [138]:
# minkowski_dist([1,3,5,7],[2,4,6,8], p=1)

In [139]:
# minkowski_dist([1,3,5,7],[2,4,6,8])

In [140]:
# minkowski_dist([1,3,5,7],[2,4,6,8], p=3)

In [141]:
# minkowski_dist([1,3,5,7],[2,4,6,8], p=4)

### 2. Write a function to calculate and return the accuracy of two vectors.

In [142]:
def accuracy(v1: np.ndarray, v2: np.ndarray):
    assert len(v1) == len(v2)
    return np.sum(v1 == v2)/np.size(v1)
    # return sum([X[i] == Y[i] for i in range(len(X))])/len(X)

In [143]:
# X = np.asarray(list(range(20)))
# Y = np.asarray(list(range(20)))
# Y[3], Y[10], X[6] = 10, 3, 2

In [144]:
# accuracy(X, Y)

### 3. Write three functions to compute: precision, recall and F1 score.

In [145]:
def precision(v1: np.ndarray, v2: np.ndarray):
    # true positive over predicted positive
    # precision measures how accurate your positive predictions are
    # which percentage of your positive predictions are correct
    # !! How many retrieved items are relevant?
    # true positives out of retrieved
    true_pos = 0
    false_pos = 0
    for i in range(len(v1)):
        if v1[i] and v2[i]:
            true_pos += 1
        if not v1[i] and v2[i]:
            false_pos += 1

    return 1.0 * true_pos / (true_pos + false_pos)

In [146]:
def recall(v1: np.ndarray, v2: np.ndarray):
    # true positive over real positive
    # recall measures how well you find all the actual positives
    # which percentage of actual positive samples were correctly classified
    # !! How many relevant items are retrieved?
    # true positives out of all actual positives
    true_pos = 0
    false_neg = 0
    
    for i in range(len(v1)):
        if v1[i] and v2[i]:
            true_pos += 1
        if v1[i] and not v2[i]:
            false_neg += 1
    return true_pos/(true_pos + false_neg)

In [147]:
def F1(v1: np.ndarray, v2: np.ndarray):
    pre = precision(v1, v2)
    rec = recall(v1, v2)
    
    return 2*(pre*rec)/(pre + rec)

In [148]:
# X = np.asarray([0, 0, 1, 1, 0, 0, 0, 1])
# Y = np.asarray([0, 0, 1, 0, 1, 0, 1, 0])
# print(accuracy(X, Y))
# print(precision(X, Y))
# print(recall(X, Y))
# print(F1(X, Y))

### 4. Write a function to compute the confusion matrix of two vectors.

In [149]:
def confusion_matrix(X: np.ndarray, Y: np.ndarray):
    true_neg = 0
    false_pos = 0
    false_neg = 0
    true_pos = 0
    
    for i in range(len(X)):
        if X[i] == Y[i] and not X[i]:
            true_neg += 1
        if X[i] != Y[i] and not X[i]:
            false_pos += 1
        if X[i] != Y[i] and X[i]:
            false_neg += 1
        if X[i] == Y[i] and X[i]:
            true_pos += 1
    return [[true_neg, false_pos], [false_neg, true_pos]]

In [150]:
# print(confusion_matrix(X, Y))

### 5. Write a function to generate the Receiver Operating Characteristic (ROC) curve.

In [151]:
import matplotlib.pyplot as plt

# need actual data
# need repr matching threshold % w/ predictions from that threshold (or FPR & TPR at least)
def predict_from_threshold(actual_data, model):
    pass

def roc(fpr_by_threshold: dict, tpr_by_threshold: dict):
    pass

### 6. Write a function to compute area under curve (AUC) for the ROC curve.

In [152]:
def auc():
    pass

## 7. Write a function to generate the precision-recall curve.

In [153]:
def precision_recall():
    pass

## 8. Implement a KNN_Classifier model class. It should have the following three methods.

#### a) __init__(self,) It’s a standard python initialization function so we can instantiate the class. Just “pass” this.

#### b) fit(self, X, Y) This method simply needs to store the relevant values as instance variables.

#### c) predict(self, X,threshold=.5) This method will use the instance variables stored by the fit method.

In [181]:
class KNN_Classifier:
    def __init__(self, n_neighbors: int, weights: str="uniform", p: int=2) -> None:
        self.n_neighbors = n_neighbors
        self.weights = weights
        self.p = p
        self.X_ = None
        self.Y_ = None

        
    def fit(self, X, Y) -> None:
        self.X_ = X
        self.Y_ = Y
    
    def predict(self, X: np.ndarray, threshold: float=.5) -> np.ndarray:
        probabilities = self.predict_proba(X)
        predictions = []
        for prob in probabilities:
            if prob >= threshold:
                predictions.append([1])
            else:
                predictions.append([0])
        return np.asarray(predictions)
    
    def predict_proba(self, X: np.ndarray) -> np.ndarray:
        probabilities = []
        # looping through every x we want to predict
        for x in X:
            distances = []
            # looping through training data rows
            for i in range(len(self.X_)):
                # find distance
                distance = minkowski_dist(x, self.X_[i], self.p)
                # weight points based on weight metric; adj made for dist of zero
                factor = 1
                if self.weights == "distance":
                    factor = 1.0/(distance + 1)
                print(distance)
                distance *= factor
                print(distance)
                print()
                # factor = 1 if self.weights == "uniform" else 1.0/(distance + 1)
                # add tuple of x, y, and distance
                distances.append((self.X_[i], self.Y_[i], distance))
            # find the k nearest neighbors
            neighbors = sorted(distances, key=lambda tup: tup[2])[:self.n_neighbors]
            # calculate and store positive class probability
            probabilities.append(sum([n[1] for n in neighbors])/self.n_neighbors)
        return np.asarray(probabilities)
    
    def get_params(self):
        return {"n_neighbors": self.n_neighbors, "weights": self.weights, "p": self.p}
    
    def set_params(self, **params: dict) -> None:
        self.n_neighbors = params.get("n_neighbors", self.n_neighbors)
        self.weights = params.get("weights", self.weights)
        self.p = params.get("p", self.p)
    

### 9. Write a function named “partition” to split your data into training and test sets. The function should take 4 arguments

In [155]:
def partition(feature: np.ndarray, target: np.ndarray, t: float, shuffle: bool=True) -> tuple:
    training_size = int(t*len(feature))
    samples = [(feature[i], target[i]) for i in range(len(feature))]
    if shuffle:
        p = np.random.permutation(len(feature))
        feature = feature[p]
        target = target[p]
    return (feature[:training_size], feature[training_size:], target[:training_size], target[training_size:])

### 10. Read in the winequality-white.csv file as a Pandas data frame.

In [156]:
# fetch dataset 
wine_quality = fetch_ucirepo(id=186) 
  
# data (as pandas dataframes) 
features = wine_quality.data.features.copy()
# features = features.drop(["density", "pH", "chlorides"], axis=1)
targets = wine_quality.data.targets.copy()

### 11. The target will be the “quality” column which represents the rating of wine and ranges from 3 to 8. You will need to convert it into a two-category variable consisting of “good” (quality > 5) & “bad” (quality <= 5). Your target vector should have 0s (representing “bad” quality wine) and 1s (representing “good” quality wine).

In [157]:
# transform quality column into binary "good" and "bad"
targets["quality"] = np.where(targets["quality"] < 5, 0, 1)


features_np = features.to_numpy()
targets_np = targets.to_numpy()

print(features)
print(targets)

      fixed_acidity  volatile_acidity  citric_acid  residual_sugar  chlorides  \
0               7.4              0.70         0.00             1.9      0.076   
1               7.8              0.88         0.00             2.6      0.098   
2               7.8              0.76         0.04             2.3      0.092   
3              11.2              0.28         0.56             1.9      0.075   
4               7.4              0.70         0.00             1.9      0.076   
...             ...               ...          ...             ...        ...   
6492            6.2              0.21         0.29             1.6      0.039   
6493            6.6              0.32         0.36             8.0      0.047   
6494            6.5              0.24         0.19             1.2      0.041   
6495            5.5              0.29         0.30             1.1      0.022   
6496            6.0              0.21         0.38             0.8      0.020   

      free_sulfur_dioxide  

In [158]:
#classifier = KNN_Classifier(n_neighbors=5, weights="distance", p=2)
#classifier.fit(features_training, target_training)
#classifier.get_params()
#b = classifier.predict(features_test)
# accuracies = []
# classifier = KNN_Classifier(n_neighbors=5, weights="distance", p=2)
# for i in range(5):
#     features_training, features_test, target_training, target_test = partition(features, targets, .8)
#     classifier.fit(features_training, target_training)
#     prediction = classifier.predict(features_test)
#     accuracies.append(accuracy(prediction, target_test))
# print(accuracies)
# print(sum(accuracies)/len(accuracies))
# classifier = KNN_Classifier(n_neighbors=5, weights="distance", p=2)
# classifier.fit(features, targets)
# predictions = classifier.predict(features)
# print(predictions)
# print(accuracy(predictions, targets))

# print(F1(prediction, targets))
# print(recall(prediction, targets))
# print(precision(prediction, targets))

### 12. Provide a table with univariate statistics of your data (mean, standard deviation, and quartiles, min, max, missing count, number of unique values).

i'm unsure of how this should be represented? we could have rows as um the feature in question? and we could also have columns as the univariate statistic. we could make separate tables for each of the features and then construct an individual table with the statistics as single column names. it would likely be a 1x7 matrix excluding the table labels. actually now that i think about it it makes more sense for the rows of the table to be each individual statistical feature


it could look something like this:

   mean | std. dev | quartiles | min | max | missing count | number of unique values
f1 
f2
f3
f4
f5

In [159]:
import scipy.stats as sp
import pandas as pd
import sklearn as sk
# .preprocessing.StandardScaler
targets_description = targets.describe(include='all')
# targets_description.loc['unique'] = len(np.unique(targets.dropna(), axis=0))
# targets_description.loc['missing'] = None
targets_description
features_description = features.describe(include='all')
# features_description.loc['unique'] = len(np.unique(features.dropna(), axis=0))
# features_description.loc['missing'] = None
features_description

#targets.describe(include = 'all')
#df["missing"] = None
#df["num unique"] = np.asarray([len(x) for x in np.unique(features, axis=0)])

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
count,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000,6497.000000
mean,7.215307,0.339666,0.318633,5.443235,0.056034,30.525319,115.744574,0.994697,3.218501,0.531268,10.491801
std,1.296434,0.164636,0.145318,4.757804,0.035034,17.749400,56.521855,0.002999,0.160787,0.148806,1.192712
min,3.800000,0.080000,0.000000,0.600000,0.009000,1.000000,6.000000,0.987110,2.720000,0.220000,8.000000
25%,6.400000,0.230000,0.250000,1.800000,0.038000,17.000000,77.000000,0.992340,3.110000,0.430000,9.500000
50%,7.000000,0.290000,0.310000,3.000000,0.047000,29.000000,118.000000,0.994890,3.210000,0.510000,10.300000
75%,7.700000,0.400000,0.390000,8.100000,0.065000,41.000000,156.000000,0.996990,3.320000,0.600000,11.300000
max,15.900000,1.580000,1.660000,65.800000,0.611000,289.000000,440.000000,1.038980,4.010000,2.000000,14.900000


### 13. Generate pair plots using the seaborn package to help identify redundant features. For any redundant features(?), report, drop, and explain your logic (w/ markdown). 

### 14. Use your “partition” function to split the data into 80% train and 20% test. 

In [160]:
features_training, features_test, target_training, target_test = partition(features_np, targets_np, .8)

### 15. Naively run your KNN_Classifier model on the training dataset with n_neighbors = 5 and using Euclidean distance.

In [161]:
classifier = KNN_Classifier(n_neighbors=5, p=2)
classifier.fit(features_training, target_training)
nonstd_predictions = classifier.predict(features_test)

#### a. Use accuracy and F1 score to compare your predictions to the expected labels.

In [162]:
print("accuracy:", accuracy(nonstd_predictions, target_test))
print("F1:", F1(nonstd_predictions, target_test))

accuracy: 0.9638461538461538
F1: 0.981561396626128


##### b. Now standardize each feature of your training set (subtract mean and divide by standard deviation) and apply trained standardization to the test set. Use the mean and standard deviation values for each feature in the training set to scale the test data (you can use sklearn.preprocessing.StandardScaler)

In [163]:
from sklearn.preprocessing import StandardScaler
# find mean and standard deviation of training set
# use mean/std dev to standardize the training set
# use that same mean and std. dev to standardize the test data

scaler = StandardScaler()

scaler.fit(features_training)
features_training_std = scaler.transform(features_training)
features_test_std = scaler.transform(features_test)

In [164]:
# print(scaler.mean_)
# print(features_training[0])
# print([a[0] for a in features_training_std])

In [165]:
# print(np.mean(features_training_std, axis=1))
# print(features_training_std[0])
# m = sum(a[0] for a in features_training_std)/len(features_training_std)
# print(m)
# print((sum([(a[0]-m)**2 for a in features_training_std])/len(features_training_std))**.5)

In [166]:
"""Just checking my work"""
# m1 = sum([a[1] for a in features_training])/len(features_training)
# st1 = (sum([(a[1]-m1)**2 for a in features_training])/len(features_training))**.5
# print(m1)
# print(st1)
# print([a[1] for a in features_training])
# print([(a[1]-m1)/st1 for a in features_training])

'Just checking my work'

##### c. Re-run the KNN_Classifier model on the standardized data, find the accuracy and F1 score with the expected labels.

In [167]:
classifier = KNN_Classifier(n_neighbors=5, p=2)
classifier.fit(features_training_std, target_training)
std_predictions = classifier.predict(features_test_std)

In [168]:
print("accuracy (standardized):", accuracy(std_predictions, target_test))
print("F1 (standardized):", F1(std_predictions, target_test))

accuracy (standardized): 0.9646153846153847
F1 (standardized): 0.9819324430479183


    Result: The model using non-standardized data performs slightly better than the standardized version. I will utilize the non-standardized data for the rest of the assignment even though standardized data (in theory) performs better than non-standardized data in most cases.

##### e. Perform a similar test for inverse distance weighting in the KNN_Classifier model and determine whether or not to use it.

In [ ]:
# with IDW

classifier = KNN_Classifier(n_neighbors=5, weights="distance", p=2)
classifier.fit(features_training, target_training)
nonstd_predictions = classifier.predict(features_test)

print("accuracy:", accuracy(nonstd_predictions, target_test))
print("F1:", F1(nonstd_predictions, target_test))

In [120]:
# without IDW

classifier = KNN_Classifier(n_neighbors=5, p=2)
classifier.fit(features_training, target_training)
nonstd_predictions = classifier.predict(features_test)

print("accuracy:", accuracy(nonstd_predictions, target_test))
print("F1:", F1(nonstd_predictions, target_test))

accuracy: 0.9676923076923077
F1: 0.983580922595778


    In this case, there is no difference in the predictions made using uniform versus distance weighting. I'll utilize distance for the remainder of this assignment since it (in theory) performs better than using uniform distance in most cases.

### 16. Repeat #15 a-d, but using a logistic regression with ‘elasticnet’ or ‘l2’ penalty (feel free to use sklearn.linear_model.LogisticRegression) 

In [125]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()
lr.fit(features_training, target_training)
lr_pred = lr.predict(features_test)

print("accuracy:", accuracy(lr_pred, target_test))
print("F1:", F1(lr_pred, target_test))

(1, 5197)
accuracy: 1261.0
F1: 0.9847715736040609


/Users/mnek/anaconda3/lib/python3.11/site-packages/sklearn/utils/validation.py:1184: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/Users/mnek/anaconda3/lib/python3.11/site-packages/sklearn/linear_model/_logistic.py:460: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 17) Evaluation of an estimator performance via cross-validation: Implement the S-fold cross validation function.

In [175]:
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

def sPartition(folds, data):
    '''Returns list of ({i:training_i}, {j: test_j}) tuples in preparation for k-fold cross validation'''
    kf = KFold(n_splits=folds)
    partitioned_data = []
    for _, (training, test) in enumerate(kf.split(data)):
        partitioned_data.append(({i:data[i] for i in training}, {j:data[j] for j in test}))
    return partitioned_data

# label means real-world result of training data
def sFold(folds, data, labels, model, model_args, error_function=mean_squared_error):
    partitioned_data = sPartition(folds, data)
    expected_labels_by_partition = []
    predicted_labels_by_partition = []

    for partition in partitioned_data:
        model_ = model(**model_args)

        x_ = list(partition[0].values())
        y_ = [labels[i] for i in partition[0].keys()]

        model_.fit(x_, y_)

        expected = [labels[i] for i in partition[1].keys()]
        predicted = model_.predict(list(partition[1].values()))

        expected_labels_by_partition.append(expected)
        predicted_labels_by_partition.append(predicted)

    avg_error = sum([error_function(expected_labels_by_partition[i], predicted_labels_by_partition[i]) for i in range(folds)])/folds

    return (expected_labels_by_partition, predicted_labels_by_partition, avg_error)


### 18) Only using the training portion of your data, use your sfold function to evaluate the performance of your model over each combination of k and distance metrics from the following sets:

        i. k=[1,5,9,11]
                b. distance = [Euclidean, Manhattan]
        ii. weights = [uniform, distance]

In [177]:
df = [["Experiment name", "k", "distance", "weights", "Average F1"]]
count = 0
for k in [1, 5, 9, 11]:
    for distance in ["Euclidean", "Manhattan"]:
        for weights in ["uniform", "distance"]:
            count += 1
            avg_error = sFold(folds=5, data=features_training, labels=target_training, model=KNN_Classifier, model_args={"n_neighbors": k, "weights": weights, "p": 1 if distance == "Manhattan" else 2}, error_function=F1)[-1]
            df.append(["Experiment #"+str(count), k, distance, weights, avg_error])
print(pd.DataFrame(df[1:], columns=df[0]))


   Experiment name   k   distance   weights  Average F1
0    Experiment #1   1  Euclidean   uniform    0.968357
1    Experiment #2   1  Euclidean  distance    0.968357
2    Experiment #3   1  Manhattan   uniform    0.969282
3    Experiment #4   1  Manhattan  distance    0.969282
4    Experiment #5   5  Euclidean   uniform    0.979363
5    Experiment #6   5  Euclidean  distance    0.979363
6    Experiment #7   5  Manhattan   uniform    0.979169
7    Experiment #8   5  Manhattan  distance    0.979169
8    Experiment #9   9  Euclidean   uniform    0.980375
9   Experiment #10   9  Euclidean  distance    0.980375
10  Experiment #11   9  Manhattan   uniform    0.980371
11  Experiment #12   9  Manhattan  distance    0.980371
12  Experiment #13  11  Euclidean   uniform    0.980378
13  Experiment #14  11  Euclidean  distance    0.980378
14  Experiment #15  11  Manhattan   uniform    0.980475
15  Experiment #16  11  Manhattan  distance    0.980475


In [179]:
df = pd.DataFrame(df[1:], columns=df[0])
df.loc[df['Average F1'].idxmax()]

Experiment name    Experiment #15
k                              11
distance                Manhattan
weights                   uniform
Average F1               0.980475
Name: 14, dtype: object

    Based on the experiment conducted, the best configuration for the model would be K=11, Manhattan distance, and inverse distance weighting.

In [180]:
# recheck work on IDW vs uniform distance bc they really shouldn't be identical??